# Print Tables 7 and 9
Table 7 and 9 show the REari calculated with T-CARS (Table7) and the comparison of REari from the observational approach to T-CARS (Table9).


In [2]:
# imports and configuration
import os
import xarray as xr
import numpy as np
import configparser

import modules.load_data as ld

def get_metrics(O,P):
    #ensure no nans in data
    idx = ~np.isnan(O)*~np.isinf(O)
    idx*= ~np.isnan(P)*~np.isinf(P)
    O = O[idx]
    P = P[idx]
    # precalculation
    N = len(O)
    N1 = 1./float(N)
    N2 = 1./(float(N)+1.)
    DELTA = P-O
    SUM = P+O
    # MBE
    MBE = N1 * np.sum(DELTA)
    # RMSE
    RMSE = np.sqrt(N2*np.sum(DELTA**2))
    # R
    ccoef = np.corrcoef(P,O)
    ccoef = ccoef[0][1]
    return MBE,RMSE,ccoef

stations = ld.dwd_stations()
stations = stations.where(stations.station!='ZG',drop=True)

models = np.array(['MRM61',
                  'MMAC',
                  'Heliosat1I',
                  'CEM',
                  'ESRA',
                  'METSTAT',
                  'SOLISsimple'])
models_label = np.array(['MRMv6.1',
                         'MMAC',
                         'Heliosat-1',
                         'CEM',
                         'ESRA',
                         'METSTAT',
                         'Solis simple'])
models_rank_global = np.array([6,8,12,17,19,23,34])
models_rank_temperate = np.array([18,9,13,28,32,26,21])

config = configparser.ConfigParser()
config.read("ConfigFile.ini")
pf = config['PATHS']['datasets']

## Prepare Dataset
REari simulated with T-CARS and the CSF method are stored in two datasets. The gridded T-CARS dataset "ECRAD" is interpolated to each of the DWD stations. The interpolated values are stored in "ECDWD". 

In [3]:
TCARS = xr.open_dataset(os.path.join(pf,"TCARS.nc"))
# bias correction
TCARS.REari_sfc.values = TCARS.REari_sfc.values+1.5
TCARS.REari_toa.values = TCARS.REari_toa.values-0.6


ECRAD = xr.open_dataset(os.path.join(pf,"REari_TCARS2DWD.nc"))
# bias correction
ECRAD.REari_sfc.values = ECRAD.REari_sfc.values+1.5
ECRAD.REari_toa.values = ECRAD.REari_toa.values-0.6


DWD = xr.open_dataset(os.path.join(pf,"REari_CSF2DWD.nc"))

# ECRAD on DWD stations
ECDWD_sfc = np.zeros((len(ECRAD.day),len(DWD.station),len(DWD.model)))*np.nan
ECDWD_toa = np.zeros((len(ECRAD.day),len(DWD.station),len(DWD.model)))*np.nan
coords = dict(day = ECRAD.day,
              station = DWD.station,
              model = DWD.model)
for s,station in enumerate(stations.station.values):
    st = stations.sel(station=station)
    EC = ECRAD.sel(station=station)
    for m,model in enumerate(models):
        ARE = DWD.ARE_csf.sel(station=station,model=model)
        ARE[ARE>=0]=np.nan
        idx = ~np.isnan(ARE.values)*~np.isnan(EC.REari_sfc.values)
        ECDWD_sfc[:,s,m] = EC.REari_sfc.where(idx)
        ECDWD_toa[:,s,m] = EC.REari_toa.where(idx)

ECDWD = xr.Dataset({'ARE_sfc':(('day','station','model'),ECDWD_sfc),
                    'ARE_toa':(('day','station','model'),ECDWD_toa)},
                   coords=coords)

## Results:

In [3]:
print("%%%%% Print Table 7:")
EC_sfc=[]
EC_toa=[]
for level in ['sfc','toa']:
    EC_seas = TCARS[f'REari_{level}'].groupby('day.season').mean(skipna=True)
    for seas in ['DJF','MAM','JJA','SON']:
        globals()[f'EC_{level}'].append(np.nanmean(EC_seas.sel(season=seas).values))
EC_toa.append(np.mean(TCARS.REari_toa.values))
EC_sfc.append(np.mean(TCARS.REari_sfc.values))
print()
print("SFC"+f" & {EC_sfc[0]:6.2f} & {EC_sfc[1]:6.2f} & {EC_sfc[2]:6.2f}  & {EC_sfc[3]:6.2f}  & {EC_sfc[4]:6.2f} \\\\")
print("TOA"+f" & {EC_toa[0]:6.2f} & {EC_toa[1]:6.2f} & {EC_toa[2]:6.2f}  & {EC_toa[3]:6.2f}  & {EC_toa[4]:6.2f} \\\\")
print("ATM"+f" & {EC_toa[0]-EC_sfc[0]:6.2f} & {EC_toa[1]-EC_sfc[1]:6.2f} & {EC_toa[2]-EC_sfc[2]:6.2f}  & {EC_toa[3]-EC_sfc[3]:6.2f}  & {EC_toa[4]-EC_sfc[4]:6.2f}\\\\")


%%%%% Print Table 7:

SFC &  -2.22 & -14.20 & -20.76  &  -5.06  & -10.62 \\
TOA &  -3.25 &  -7.87 & -10.23  &  -4.53  &  -6.49 \\
ATM &  -1.03 &   6.33 &  10.53  &   0.53  &   4.13\\


In [4]:
## unify nans for different models
idx_sfc = np.ones((ECDWD.day.size,ECDWD.station.size))
for model in ECDWD.model.values:
    idx_sfc-=np.isnan(ECDWD.sel(model=model).ARE_sfc.values)

idx_sfc[idx_sfc<0]=0
idx_sfc=idx_sfc.astype(bool)

## make Table
print("%%%%%% Print Table 9 :")
print()

msort=[0,4,2,6,3,1,5]
for i,model in enumerate(zip(models[msort],
                             models_label[msort])):
    model,mlabel=model
    AREcsf=[]
    dwd = DWD.sel(model=model).where(idx_sfc).mean(dim='station',skipna=True)
    ecd = ECDWD.sel(model=model).where(idx_sfc).mean(dim='station',skipna=True)
    dwdseas = dwd.groupby('day.season').mean(skipna=True)
    for seas in ['DJF','MAM','JJA','SON']:
        AREcsf.append(dwdseas.ARE_csf.sel(season=seas).values)
    AREcsf.append(dwd.ARE_csf.mean(dim='day',skipna=True).values)
    EARE = ecd.ARE_sfc.values
    DARE = dwd.ARE_csf.values
    mbe,rmse,corr = get_metrics(EARE,DARE)
    print(f"{mlabel:12s} & {models_rank_temperate[msort][i]:2d} & {AREcsf[4]:6.2f} & {mbe:6.1f} & {rmse:6.1f} & {corr:6.2f} \\\\")
print("\\middlehline")

AREcsf = []
ECsfc = []
dwd = DWD.mean(dim='model',skipna=True).where(idx_sfc).mean(dim=['station'],skipna=True)
ecd = ECDWD.mean(dim='model',skipna=True).where(idx_sfc).mean(dim=['station'],skipna=True)
dwdseas = dwd.groupby('day.season').mean(skipna=True)
ecdseas = ecd.groupby('day.season').mean(skipna=True)
for seas in ['DJF','MAM','JJA','SON']:
    AREcsf.append(dwdseas.ARE_csf.sel(season=seas).values)
    ECsfc.append(ecdseas.ARE_sfc.sel(season=seas).values)
AREcsf.append(dwd.ARE_csf.mean(dim='day',skipna=True).values)
ECsfc.append(ecd.ARE_sfc.mean(dim='day',skipna=True).values)
EARE = ecd.ARE_sfc.values
DARE = dwd.ARE_csf.values
mbe,rmse,corr = get_metrics(EARE,DARE)

print(f"CSM mean & & {AREcsf[4]:6.2f} & {mbe:6.1f} & {rmse:6.1f} & {corr:6.2f} \\\\")
# print("\\middlehline")
# print("T--CARS "+f" &  - & {ECsfc[0]:6.2f} & {ECsfc[1]:6.2f} & {ECsfc[2]:6.2f}  & {ECsfc[3]:6.2f}  & {ECsfc[4]:6.2f} &&&\\\\")

%%%%%% Print Table 9 :

MRMv6.1      & 18 & -13.11 &   -2.1 &    6.3 &   0.72 \\
ESRA         & 32 & -15.18 &   -4.2 &    6.6 &   0.77 \\
Heliosat-1   & 13 & -11.67 &   -0.7 &    5.5 &   0.74 \\
Solis simple & 21 & -11.14 &   -0.1 &    6.4 &   0.64 \\
CEM          & 28 & -24.57 &  -13.6 &   14.6 &   0.77 \\
MMAC         &  9 &  -8.23 &    2.8 &    7.6 &   0.54 \\
METSTAT      & 26 &  -5.35 &    5.6 &    8.7 &   0.60 \\
\middlehline
CSM mean & & -12.75 &   -1.8 &    5.8 &   0.75 \\


In [5]:
## unify nans for different models
idx_sfc = np.ones((ECDWD.day.size,ECDWD.station.size))
for model in ECDWD.model.values:
    idx_sfc-=np.isnan(ECDWD.sel(model=model).ARE_sfc.values)

idx_sfc[idx_sfc<0]=0
idx_sfc=idx_sfc.astype(bool)

## make Table
print("%%%%%% Print Table 9 :")
print()

msort=[0,4,2,6,3,1,5]
for i,model in enumerate(zip(models[msort],
                             models_label[msort])):
    model,mlabel=model
    AREcsf=[]
    dwd = DWD.sel(model=model).where(idx_sfc).mean(dim='station',skipna=True)
    ecd = ECDWD.sel(model=model).where(idx_sfc).mean(dim='station',skipna=True)
    dwdseas = dwd.groupby('day.season').mean(skipna=True)
    for seas in ['DJF','MAM','JJA','SON']:
        AREcsf.append(dwdseas.ARE_csf.sel(season=seas).values)
    AREcsf.append(dwd.ARE_csf.mean(dim='day',skipna=True).values)
    EARE = ecd.ARE_sfc.values
    DARE = dwd.ARE_csf.values
    mbe,rmse,corr = get_metrics(EARE,DARE)
    print(f"& {mlabel:12s} & {models_rank_temperate[msort][i]:2d} & {AREcsf[0]:6.2f} & {AREcsf[1]:6.2f} & {AREcsf[2]:6.2f}  & {AREcsf[3]:6.2f}  & {AREcsf[4]:6.2f} & {mbe:6.1f} & {rmse:6.1f} & {corr:6.2f} \\\\")
print("\\middlehline")

AREcsf = []
ECsfc = []
dwd = DWD.mean(dim='model',skipna=True).where(idx_sfc).mean(dim=['station'],skipna=True)
ecd = ECDWD.mean(dim='model',skipna=True).where(idx_sfc).mean(dim=['station'],skipna=True)
dwdseas = dwd.groupby('day.season').mean(skipna=True)
ecdseas = ecd.groupby('day.season').mean(skipna=True)
for seas in ['DJF','MAM','JJA','SON']:
    AREcsf.append(dwdseas.ARE_csf.sel(season=seas).values)
    ECsfc.append(ecdseas.ARE_sfc.sel(season=seas).values)
AREcsf.append(dwd.ARE_csf.mean(dim='day',skipna=True).values)
ECsfc.append(ecd.ARE_sfc.mean(dim='day',skipna=True).values)
EARE = ecd.ARE_sfc.values
DARE = dwd.ARE_csf.values
mbe,rmse,corr = get_metrics(EARE,DARE)

print(f"CSM mean & & {AREcsf[0]:6.2f} & {AREcsf[1]:6.2f} & {AREcsf[2]:6.2f}  & {AREcsf[3]:6.2f}  & {AREcsf[4]:6.2f} & {mbe:6.1f} & {rmse:6.1f} & {corr:6.2f} \\\\")
# print("\\middlehline")
# print("T--CARS "+f" &  - & {ECsfc[0]:6.2f} & {ECsfc[1]:6.2f} & {ECsfc[2]:6.2f}  & {ECsfc[3]:6.2f}  & {ECsfc[4]:6.2f} &&&\\\\")

%%%%%% Print Table 9 :

& MRMv6.1      & 18 &  -7.72 & -14.18 & -15.83  & -10.09  & -13.11 &   -2.1 &    6.3 &   0.72 \\
& ESRA         & 32 &  -5.98 & -15.67 & -21.31  &  -9.35  & -15.18 &   -4.2 &    6.6 &   0.77 \\
& Heliosat-1   & 13 &  -4.37 & -11.86 & -16.97  &  -6.60  & -11.67 &   -0.7 &    5.5 &   0.74 \\
& Solis simple & 21 &  -7.01 & -13.21 & -12.70  &  -8.05  & -11.14 &   -0.1 &    6.4 &   0.64 \\
& CEM          & 28 & -13.44 & -26.15 & -30.90  & -17.99  & -24.57 &  -13.6 &   14.6 &   0.77 \\
& MMAC         &  9 &  -5.71 &  -9.08 &  -9.44  &  -6.49  &  -8.23 &    2.8 &    7.6 &   0.54 \\
& METSTAT      & 26 &  -2.21 &  -5.08 &  -7.50  &  -3.84  &  -5.35 &    5.6 &    8.7 &   0.60 \\
\middlehline
& CSM total & &  -6.63 & -13.60 & -16.38  &  -8.91  & -12.75 &   -1.8 &    5.8 &   0.75 \\
\middlehline
& T-CARS  &  - &  -2.31 & -11.32 & -17.62  &  -4.36  & -11.00 &&&\\
